In [244]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import re
import pandas as pd
from sqlalchemy import create_engine

In [301]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [246]:
tournament_history_url = 'https://www.sports-reference.com/cbb/postseason/'
browser.visit(tournament_history_url)

ncaa_soup = bs(browser.html,'html.parser')
history_table = ncaa_soup.find('div', id='all_ncaa-tournament-history')
history_list = history_table.find('table', id='ncaa-tournament-history')
tournaments = history_list.find('tbody').find_all('tr')

In [247]:
tournament_urls = []

for tournament in tournaments:
    try:
        year = tournament.find('a').text[:4]
        int(year)
    except:
        print('not a valid year, continuing')
    else:
        print(year)
        url = 'https://www.sports-reference.com'+tournament.find('a')['href']
        info = {'year': year,
                'url': url}
        tournament_urls.append(info)

2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991
1990
1989
1988
1987
1986
1985
1984
1983
1982
1981
1980
1979
1978
1977
1976
1975
1974
1973
1972
1971
1970
1969
1968
1967
1966
1965
1964
1963
1962
1961
1960
1959
1958
1957
1956
1955
1954
1953
1952
1951
1950
1949
1948
1947
1946
1945
1944
1943
1942
1941
1940
1939
not a valid year, continuing


In [248]:
def get_game_info(game_soup, game_list, tournament_year, tournament_round):
    location = game_soup.find('span', recursive=False).text.split('at ')[1]
    
    winner = game_soup.find('div', class_='winner')
    winner_name = winner.find('a').text
    winner_seed = winner.find('span').text
    winner_score = winner.find_all('a')[1].text
    
    loser = game_soup.find('div', attrs = {'class': None})
    loser_name = loser.find('a').text
    loser_seed = loser.find('span').text
    loser_score = loser.find_all('a')[1].text
    
    game_info = {'year': tournament_year,
                'round': tournament_round,
                'location': location,
                'winner': winner_name,
                'winner_seed': winner_seed,
                'winner_score': winner_score,
                'loser': loser_name,
                'loser_seed': loser_seed,
                'loser_score': loser_score}
    
    game_list.append(game_info)

In [250]:
teams = []
games = []

for tournament in tournament_urls:
    year = tournament['year']
    
    if int(year) >= 1993:
        
        browser.visit(tournament['url'])
        tournament_soup = bs(browser.html,'html.parser')

        brackets = tournament_soup.find('div', id='brackets').find_all('div', class_='team16')

        for bracket in brackets:
            rounds = bracket.find_all('div', class_='round')
            
            ########## ROUND 1 ##########
            rd_1 = rounds[0]
            rd_1_games = rd_1.find_all('div', recursive=False)

            for game in rd_1_games:
                gameInfo(game, games, year,1)
                rd_1_teams = game.find_all('div')

                for team in rd_1_teams:
                    team_name = team.find('a').text
                    team_seed = team.find('span').text
                    team_url = 'https://www.sports-reference.com'+team.find('a')['href']

                    team_info = {'year': year,
                                'name': team_name,
                                'seed': team_seed,
                                'url': team_url}

                    teams.append(team_info)
            
            ########## ROUND 2 ##########
            rd_2 = rounds[1]
            rd_2_games = rd_2.find_all('div', recursive=False)
            
            for game in rd_2_games:
                get_game_info(game, games, year, 2)
            
            ########## ROUND 3 ##########
            rd_3 = rounds[2]
            rd_3_games = rd_3.find_all('div', recursive=False)
            
            for game in rd_3_games:
                get_game_info(game, games, year, 3)
            
            ########## ROUND 4 ##########    
            rd_4 = rounds[3]
            rd_4_games = rd_4.find_all('div', recursive=False)
            
            for game in rd_4_games:
                get_game_info(game, games, year, 4)
                
            ########## FINAL FOUR ########## 
            ff_bracket = tournament_soup.find('div', id='brackets').find('div', class_='team4')
            ff_rounds = ff_bracket.find_all('div', recursive=False)
            
            ff = ff_rounds[0]
            ff_games = ff.find_all('div', recursive=False)
            
            for game in ff_games:
                get_game_info(game, games, year, 5)
                
            ########## FINALS ##########
            finals = ff_rounds[1]
            finals_game = finals.find('div')
            
            get_game_info(finals_game, games, year, 6)

In [251]:
def ordinal_to_number(ordinal):
    number = re.sub('[stndrdh]','',ordinal)
    return number

In [261]:
for team in teams:
    print(team['year'])

    team_url = team['url']
    browser.visit(team['url'])
    team_soup = bs(browser.html,'html.parser')
    
    team_meta_table = team_soup.find('div', attrs={'data-template': 'Partials/Teams/Summary'})
    team_meta_info = team_meta_table.find_all('p')
    
    for info in team_meta_info:
        if 'SRS' in info.text:
            team['team_SRS'] = re.sub('[SRS: ]','',info.text).split('(')[0]
            team['team_rank_SRS'] = ordinal_to_number(re.sub('[SRS: ]','',info.text).split('of')[0].split('(')[1])
        elif 'SOS' in info.text:
            team['team_SOS'] = re.sub('[SOS: ]','',info.text).split('(')[0]
            team['team_rank_SOS'] = ordinal_to_number(re.sub('[SOS: ]','',info.text).split('of')[0].split('(')[1])

    stats_table = team_soup.find('table', id='team_stats')
    stat_table_rows = stats_table.find('tbody').find_all('tr')

    team_stats = stat_table_rows[0].find_all('td')
    team_rank = stat_table_rows[1].find_all('td')
    opp_stats = stat_table_rows[2].find_all('td')
    opp_rank = stat_table_rows[3].find_all('td')

    for stat in team_stats:
        label = 'team_'+stat['data-stat']
        value = stat.text

        team[label] = value

    for stat in team_rank:
        label = 'team_rank_'+stat['data-stat']
        value = ordinal_to_number(stat.text)

        team[label] = value

    for stat in opp_stats:
        label = 'opp_'+stat['data-stat']
        value = stat.text

        team[label] = value 

    for stat in opp_rank:
        label = 'opp_rank_'+stat['data-stat']
        value = ordinal_to_number(stat.text)

        team[label] = value

2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2015
2015
2015
2015
2015
2015
2015
2015


1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993


In [318]:
school_info = []
schools_url = 'https://www.sports-reference.com/cbb/schools/'
    
browser.visit(schools_url)
schools_soup = bs(browser.html,'html.parser')

school_table = schools_soup.find('table', id='schools')
school_rows = school_table.find('tbody').find_all('tr', attrs={'class': None})

for school in school_rows:
    school_name = school.find('td', attrs={'data-stat': 'school_name'}).find('a').text
    school_url = 'https://www.sports-reference.com'+school.find('td', attrs = {'data-stat': 'school_name'}).find('a')['href']
    school_location = school.find('td', attrs={'data-stat': 'location'}).text
    
    browser.visit(school_url)
    school_soup = bs(browser.html,'html.parser')
    school_logo = school_soup.find('img', class_='teamlogo')['src']
    
    school_data = {'name': school_name,
                  'url': school_url,
                  'location': school_location,
                  'logo': school_logo}
    
    school_info.append(school_data)

In [324]:
for team in teams:
    school_url = team['url'][:-9]
    
    for school in school_info:
        if school['url'] == school_url:
            new_name = {'name': team['name']}
            school.update(new_name)

In [325]:
teams_df = pd.DataFrame(teams)
games_df = pd.DataFrame(games)
schools_df = pd.DataFrame(school_info)

In [327]:
engine = create_engine("sqlite:///db/ncaa_history.sqlite")
teams_df.to_sql('teams_all', con=engine, if_exists='replace', index=False)
games_df.to_sql('games_all', con=engine, if_exists='replace', index=False)
schools_df.to_sql('schools', con=engine, if_exists='replace', index=False)